In [1]:
!pip install imbalanced-learn


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks


In [4]:
df = pd.read_csv("Creditcard_data.csv")
X = df.drop("Class", axis=1)
y = df["Class"]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [6]:
samplers = {
    "Sampling1": RandomUnderSampler(),
    "Sampling2": RandomOverSampler(),
    "Sampling3": SMOTE(),
    "Sampling4": TomekLinks(),
    "Sampling5": SMOTETomek()
}


In [16]:
models = {
    "M1": make_pipeline(StandardScaler(), LogisticRegression(max_iter=5000)),
    "M2": DecisionTreeClassifier(),
    "M3": RandomForestClassifier(),
    "M4": make_pipeline(StandardScaler(), KNeighborsClassifier()),
    "M5": make_pipeline(StandardScaler(), SVC())
}


In [17]:
results = {}

for s_name, sampler in samplers.items():
    X_res, y_res = sampler.fit_resample(X_train, y_train)

    results[s_name] = []

    for m_name, model in models.items():
        model.fit(X_res, y_res)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds) * 100
        results[s_name].append(round(acc, 2))


In [18]:
table = pd.DataFrame(results, index=models.keys())
table


,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1,41.29,90.97,91.61,98.71,90.97
M2,62.58,98.71,95.48,97.42,98.71
M3,71.61,99.35,98.71,98.71,99.35
M4,78.06,96.13,94.19,98.71,92.90
M5,65.16,94.84,95.48,98.71,94.84


In [19]:
table.idxmax(axis=1)


,0
M1,Sampling4
M2,Sampling2
M3,Sampling2
M4,Sampling4
M5,Sampling4
